## Webscraping
Iniciaremos o projeto realizando a raspagem da web, processo automatizado de extração de dados da web, no qual coletaremos as notícias relacionadas ao Liberation Day e às tarifas impostas pelo presidente Donald Trump divulgadas pelos principais veículos de notícias, como Brasil de Fato e Agência Brasil.

In [3]:
#Primeiro, precisamos instalar algumas bibliotecas 
#%pip install pandas numpy requests bs4 scrapy tqdm


[notice] A new release of pip is available: 24.1.2 -> 25.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [5]:
#Agora, precisamos importamos as bibliotecas para o arquivo
import pandas as pd
import numpy as np
import datetime as dt
import requests
from bs4 import BeautifulSoup
import scrapy
import random
import time
from urllib.parse import urljoin
import re
from tqdm import tqdm
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
import time
import random
import pyautogui

### Extração Automatizada de URLs via Web Scraping no Google
Aqui precisamos o código desenvolvido tem como objetivo automatizar a busca de sites oficiais ou relevantes relacionados a uma lista de nomes, utilizando técnicas de web scraping com o Google. A lógica é dividida em duas funções principais: a primeira, get_url_from_google, realiza uma busca no Google baseada no nome de uma entidade presente em um DataFrame (base), simulando o acesso de um navegador real para evitar bloqueios. Ela analisa os resultados da busca e tenta extrair a URL mais apropriada com base em padrões definidos. A segunda função, pega_url_de_lista_com_timesleep, aplica esse processo para um intervalo de índices da base de dados, realizando pausas aleatórias entre as requisições para respeitar os limites dos servidores e evitar bloqueios por excesso de acessos. O resultado final é um dicionário associando cada nome a uma ou mais URLs encontradas, o que permite enriquecer os dados com links úteis de forma automatizada, segura e eficiente.

## 1º Jornal Online - Agência Brasil

In [7]:
def get_url_from_agencia_brasil(index):
	

	# Configuração de headers para simular um navegador
	user_agents = [
    "Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:122.0) Gecko/20100101 Firefox/122.0",
    "Mozilla/5.0 (Macintosh; Apple M1; Mac OS X 13_4) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/122.0.0.0 Safari/537.36",
    "Mozilla/5.0 (X11; Fedora; Linux x86_64; rv:124.0) Gecko/20100101 Firefox/124.0",
    "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/121.0.0.0 Safari/537.36 Edg/121.0.2277.128",
    "Mozilla/5.0 (Macintosh; Intel Mac OS X 13_3_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/121.0.6167.185 Safari/537.36 OPR/98.0.4759.82",
    "Mozilla/5.0 (iPad; CPU OS 17_2 like Mac OS X) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/17.0 Mobile/15E148 Safari/604.1",
    "Mozilla/5.0 (Linux; Android 14; Pixel 7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/123.0.6312.107 Mobile Safari/537.36",
    "Mozilla/5.0 (Linux; Android 14; SM-S911B) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/123.0.6312.86 Mobile Safari/537.36",
    "Mozilla/5.0 (iPhone; CPU iPhone OS 17_3_1 like Mac OS X) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/17.0 Mobile/15E148 Safari/604.1",
    "Mozilla/5.0 (Linux; Android 13; ONEPLUS A6013) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/123.0.6312.107 Mobile Safari/537.36",
    "Mozilla/5.0 (X11; CrOS x86_64 15393.44.0) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/123.0.6312.86 Safari/537.36",
    "Mozilla/5.0 (Android 14; Mobile; rv:124.0) Gecko/124.0 Firefox/124.0",
	]

	headers = {"User-Agent": random.choice(user_agents),
			"Accept-Language": "pt-BR,pt;q=0.9",
                "Accept-Encoding": "gzip, deflate, br",
                "Connection": "keep-alive"
            }

	# Fazer a requisição ao Google
	def main(index):
		query = "+".join(index.split())
		lista = []
		for i in range(1, 50):
				
			search_url = f"https://busca.ebc.com.br/nodes?page={i}utf8=%E2%9C%93&q={query}&site_id=agenciabrasil"
			response = requests.get(search_url, headers=headers)
			soup = BeautifulSoup(response.text, "html.parser")
			links = encontra_links_do_html(soup)
			#print(links)
			lista.extend(links)
		# Break se não achar mais links (opcional)
			if not links:
				break

		return lista
	
	def encontra_links_do_html(soup):
		anchors = soup.find_all("a")
		links_encontrados = set()  # usando set para evitar duplicatas

		for tag in anchors:
			href = tag.get("href")
			if href and href.startswith("http://agenciabrasil"):
				links_encontrados.add(href.strip())

		return list(links_encontrados)
			
	return main(index)

def pega_url_de_lista_com_timesleep_agencia_brasil(palavras_chave):
	fim = dict()
	for chave in palavras_chave:
		url = get_url_from_agencia_brasil(chave)
		#print(f"URL encontrada para '{site}': {url}")
		fim[chave] = url
		# Pausa aleatória entre as requisições
		time.sleep(random.uniform(3, 15))
	return fim

palavras_chave = ['tarifas', 'tarifas Trump', 'tarifas EUA']

a = pega_url_de_lista_com_timesleep_agencia_brasil(palavras_chave)
a

{'tarifas': ['http://agenciabrasil.ebc.com.br/economia/noticia/2025-04/dolar-cai-pela-sexta-vez-seguida-e-fecha-em-r-568',
  'http://agenciabrasil.ebc.com.br/internacional/noticia/2025-04/canadenses-vao-urnas-em-eleicao-dominada-por-preocupacoes-com-trump',
  'http://agenciabrasil.ebc.com.br/internacional/noticia/2025-04/california-contesta-em-tribunal-politica-protecionista-de-trump',
  'http://agenciabrasil.ebc.com.br/economia/noticia/2025-04/inflacao-desacelera-para-todas-faixas-de-renda-em-marco',
  'http://agenciabrasil.ebc.com.br/geral/noticia/2025-04/tarifaco-e-fundo-ambiental-sao-prioridades-de-chanceleres-do-brics',
  'http://agenciabrasil.ebc.com.br/economia/noticia/2025-04/bc-esta-tateando-ajuste-da-selic-diante-de-guerra-de-tarifas',
  'http://agenciabrasil.ebc.com.br/economia/noticia/2025-04/haddad-afirma-que-eua-sao-incoerentes-ao-tarifar-o-brasil',
  'http://agenciabrasil.ebc.com.br/geral/noticia/2025-04/marina-quer-avancar-debate-sobre-justica-climatica',
  'http://agen

In [8]:
def extrai_dados_noticia(url):
    user_agents = [
        "Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:122.0) Gecko/20100101 Firefox/122.0",
        "Mozilla/5.0 (Macintosh; Apple M1; Mac OS X 13_4) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/122.0.0.0 Safari/537.36",
        "Mozilla/5.0 (X11; Fedora; Linux x86_64; rv:124.0) Gecko/20100101 Firefox/124.0",
        "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/121.0.0.0 Safari/537.36 Edg/121.0.2277.128",
        "Mozilla/5.0 (Macintosh; Intel Mac OS X 13_3_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/121.0.6167.185 Safari/537.36 OPR/98.0.4759.82",
        "Mozilla/5.0 (iPad; CPU OS 17_2 like Mac OS X) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/17.0 Mobile/15E148 Safari/604.1",
        "Mozilla/5.0 (Linux; Android 14; Pixel 7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/123.0.6312.107 Mobile Safari/537.36",
        "Mozilla/5.0 (Linux; Android 14; SM-S911B) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/123.0.6312.86 Mobile Safari/537.36",
        "Mozilla/5.0 (iPhone; CPU iPhone OS 17_3_1 like Mac OS X) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/17.0 Mobile/15E148 Safari/604.1",
        "Mozilla/5.0 (Linux; Android 13; ONEPLUS A6013) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/123.0.6312.107 Mobile Safari/537.36",
        "Mozilla/5.0 (X11; CrOS x86_64 15393.44.0) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/123.0.6312.86 Safari/537.36",
        "Mozilla/5.0 (Android 14; Mobile; rv:124.0) Gecko/124.0 Firefox/124.0",
    ]

    headers = {
        "User-Agent": random.choice(user_agents),
        "Connection": "keep-alive"
    }

    response = requests.get(url, headers=headers)

    soup = BeautifulSoup(response.text, "html.parser")

    # Título
    titulo_tag = soup.find("h1")
    print(titulo_tag)
    titulo = titulo_tag.get_text(strip=True) if titulo_tag else "Título não encontrado"

    # Data de publicação atualizada
    data_tag = soup.find("div", class_="data")
    if data_tag:
        data_texto = data_tag.get_text(strip=True)
        data_publicacao = data_texto.replace("Publicado em", "").strip()
    else:
        data_publicacao = "Data não encontrada"

    # Conteúdo principal atualizado
    conteudo_tag = soup.find("div", class_="conteudo-noticia")
    if conteudo_tag:
        paragrafos = conteudo_tag.find_all("p")
        conteudo = "\n".join(p.get_text(strip=True) for p in paragrafos if p.get_text(strip=True))
    else:
        conteudo = "Conteúdo não encontrado"

    return {
        "titulo": titulo,
        "data_publicacao": data_publicacao,
        "conteudo": conteudo
    }



In [9]:
extrai_dados_noticia("https://agenciabrasil.ebc.com.br/direitos-humanos/noticia/2025-04/fruto-da-mobilizacao-indigena-apib-completa-20-anos-de-existencia")

<h1 class="titulo-materia">
        Fruto da mobilização indígena, Apib completa 20 anos       </h1>


{'titulo': 'Fruto da mobilização indígena, Apib completa 20 anos',
 'data_publicacao': '19/04/2025 - 10:10',
 'conteudo': 'Principal instância de mobilização dos povos originários, a Articulação dos Povos Indígenas do Brasil (Apib) completa duas décadas em 2025. Criada durante o segundo Acampamento Terra Livre (ATL), evento que, em 2025, chegou a sua 21ª edição, integrando o chamado Abril Indígena, a entidade tornou-se uma referência nacional, promovendo a união entre as centenas de etnias que habitam o país.\nÀs vésperas do Dia dos Povos Indígenas, data que promove a diversidade e a riqueza das culturas dos povos originários, chamando a atenção para a luta dos quase 1,7 milhão de brasileiros que, conforme o Instituto Brasileiro de Geografia e Estatística (IBGE), se declaram indígenas, aAgência Brasilentrevistou duas lideranças do movimento que participaram ativamente da criação da Apib.\nO hoje chefe da Assessoria de Participação Social e Diversidade do Ministério dos Povos Indígenas,

In [ ]:
# Mantem apenas os links que possuem '2025-04', '2025-03' ou '2025-02' na URL
def filtra_links_por_data(links):
    links_filtrados = []
    for link in links:
        if re.search(r'2025-(04|03|02)', link):
            links_filtrados.append(link)
    return links_filtrados


for k in a.keys():
    a[k] = filtra_links_por_data(a[k])

132

In [21]:
### Estrutura os dados em dicionários ###
b_dicionario = dict()
for b in list(a.keys()):
    c_dicionario = dict()
    for c in list(a.values())[0]:
        c_dicionario[c] = extrai_dados_noticia(c)
    b_dicionario[b] = c_dicionario


<h1 class="titulo-materia">
        Dólar cai pela sexta vez seguida e fecha em R$ 5,68      </h1>
<h1 class="titulo-materia">
        Canadenses vão às urnas em eleição dominada por preocupações com Trump      </h1>
<h1 class="titulo-materia">
        Califórnia contesta em tribunal política protecionista de Trump      </h1>
<h1 class="titulo-materia">
        Inflação desacelera para todas as faixas de renda em março      </h1>
<h1 class="titulo-materia">
        Tarifaço e fundo ambiental são prioridades de chanceleres do Brics      </h1>
<h1 class="titulo-materia">
        BC está "tateando" ajuste da Selic diante de guerra de tarifas      </h1>
<h1 class="titulo-materia">
        Haddad afirma que EUA são incoerentes ao tarifar o Brasil      </h1>
<h1 class="titulo-materia">
        Marina quer avançar debate sobre justiça climática       </h1>
<h1 class="titulo-materia">
        China vai criticar EUA na ONU por intimidação e guerra comercial      </h1>
<h1 class="titulo-materia"

In [22]:
# Transformar b_dicionario em uma lista de linhas para o DataFrame
linhas = []
for chave_principal, sub_dicionario in b_dicionario.items():
    for url, dados in sub_dicionario.items():
        linha = {
            "Palavra-chave": chave_principal,
            "URL": url,
            "Título": dados.get("titulo", "Título não encontrado"),
            "Data de Publicação": dados.get("data_publicacao", "Data não encontrada"),
            "Conteúdo": dados.get("conteudo", "Conteúdo não encontrado"),
        }
        linhas.append(linha)

# Criar o DataFrame
df = pd.DataFrame(linhas)

# Exibir o DataFrame
print(df)

    Palavra-chave                                                URL  \
0         tarifas  http://agenciabrasil.ebc.com.br/economia/notic...   
1         tarifas  http://agenciabrasil.ebc.com.br/internacional/...   
2         tarifas  http://agenciabrasil.ebc.com.br/internacional/...   
3         tarifas  http://agenciabrasil.ebc.com.br/economia/notic...   
4         tarifas  http://agenciabrasil.ebc.com.br/geral/noticia/...   
..            ...                                                ...   
541   tarifas EUA  http://agenciabrasil.ebc.com.br/economia/notic...   
542   tarifas EUA  http://agenciabrasil.ebc.com.br/internacional/...   
543   tarifas EUA  http://agenciabrasil.ebc.com.br/politica/notic...   
544   tarifas EUA  http://agenciabrasil.ebc.com.br/internacional/...   
545   tarifas EUA  http://agenciabrasil.ebc.com.br/internacional/...   

                                                Título  Data de Publicação  \
0    Dólar cai pela sexta vez seguida e fecha em R$...  2

In [ ]:
### Impostação para o Excel ###

import openpyxl
#df.to_excel("noticias_agencia_brasil.xlsx", index=False, engine='openpyxl')

## 2º Jornal Online - Brasil de Fato

In [12]:
def get_url_from_brasil_de_fato(index):
	

	# Configuração de headers para simular um navegador
	user_agents = [
    "Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:122.0) Gecko/20100101 Firefox/122.0",
    "Mozilla/5.0 (Macintosh; Apple M1; Mac OS X 13_4) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/122.0.0.0 Safari/537.36",
    "Mozilla/5.0 (X11; Fedora; Linux x86_64; rv:124.0) Gecko/20100101 Firefox/124.0",
    "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/121.0.0.0 Safari/537.36 Edg/121.0.2277.128",
    "Mozilla/5.0 (Macintosh; Intel Mac OS X 13_3_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/121.0.6167.185 Safari/537.36 OPR/98.0.4759.82",
    "Mozilla/5.0 (iPad; CPU OS 17_2 like Mac OS X) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/17.0 Mobile/15E148 Safari/604.1",
    "Mozilla/5.0 (Linux; Android 14; Pixel 7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/123.0.6312.107 Mobile Safari/537.36",
    "Mozilla/5.0 (Linux; Android 14; SM-S911B) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/123.0.6312.86 Mobile Safari/537.36",
    "Mozilla/5.0 (iPhone; CPU iPhone OS 17_3_1 like Mac OS X) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/17.0 Mobile/15E148 Safari/604.1",
    "Mozilla/5.0 (Linux; Android 13; ONEPLUS A6013) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/123.0.6312.107 Mobile Safari/537.36",
    "Mozilla/5.0 (X11; CrOS x86_64 15393.44.0) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/123.0.6312.86 Safari/537.36",
    "Mozilla/5.0 (Android 14; Mobile; rv:124.0) Gecko/124.0 Firefox/124.0",
	]

	headers = {"User-Agent": random.choice(user_agents),
			"Accept-Language": "pt-BR,pt;q=0.9",
			"Accept-Encoding": "gzip, deflate",  # tira o 'br' pois ele estava dando problema na formatação
			"Connection": "keep-alive"
            }

	# Fazer a requisição ao Google
	def main(index):
		query = "".join(index.split())
		lista = []
		for i in range(1, 22):	
			search_url = f"https://www.brasildefato.com.br/page/{i}/?s={query}"
			response = requests.get(search_url, headers=headers)
			response.encoding = response.apparent_encoding
			soup = BeautifulSoup(response.text, "html.parser")
			links = encontra_links_do_html(soup)
			lista.extend(links) 
		
		return lista
	

	def encontra_links_do_html(soup):
		main_data = soup.find_all("a", href=True)  # pega só <a> com href
		link_sites_encontrados = []

		for link in tqdm(main_data, desc="Links"):
			href = link['href']
			if href.startswith("https://www.brasildefato.com.br/2025") and href not in link_sites_encontrados:
				link_sites_encontrados.append(href)

		return link_sites_encontrados	
			
	return main(index)

def pega_url_de_lista_com_timesleep_brasil_de_fato(palavras_chave):
	fim = dict()
	for chave in tqdm(palavras_chave, desc="Scraping"):
		url = get_url_from_brasil_de_fato(chave)
		#print(f"URL encontrada para '{site}': {url}")
		fim[chave] = url
		# Pausa aleatória entre as requisições
		time.sleep(random.uniform(3, 15))
	
	return fim

palavras_chave = ['tarifas+trump']

d = pega_url_de_lista_com_timesleep_brasil_de_fato(palavras_chave)
print(d)


Links: 100%|█████████████████████████████████████████████████████████████████████| 177/177 [00:00<00:00, 177563.22it/s]

Links: 100%|█████████████████████████████████████████████████████████████████████████████████| 180/180 [00:00<?, ?it/s]

Links: 100%|█████████████████████████████████████████████████████████████████████████████████| 179/179 [00:00<?, ?it/s]

Links: 100%|█████████████████████████████████████████████████████████████████████| 179/179 [00:00<00:00, 180432.69it/s]

Links: 100%|█████████████████████████████████████████████████████████████████████| 179/179 [00:00<00:00, 179698.52it/s]

Links: 100%|█████████████████████████████████████████████████████████████████████| 181/181 [00:00<00:00, 180025.85it/s]

Links: 100%|█████████████████████████████████████████████████████████████████████████████████| 179/179 [00:00<?, ?it/s]

Links: 100%|█████████████████████████████████████████████████████████████████████| 159/159 [00:00<00:00, 159239.34it/s]

Links: 100%|████████████████████

{'tarifas+trump': ['https://www.brasildefato.com.br/2025/04/14/guerra-comercial-russia-tem-riscos-e-oportunidades-com-tarifas-de-trump/', 'https://www.brasildefato.com.br/2025/04/13/china-pede-fim-de-tarifas-reciprocas-de-trump-eua-dao-prazo-para-inicio-de-taxas-sobre-semicondutores/', 'https://www.brasildefato.com.br/2025/04/10/venezuela-aprova-decreto-de-emergencia-economica-para-combater-tarifas-de-trump/', 'https://www.brasildefato.com.br/2025/04/09/lula-critica-tarifas-arbitrarias-de-trump-em-discurso-na-celac-guerras-comerciais-nao-tem-vencedores/', 'https://www.brasildefato.com.br/2025/04/08/nao-aceitaremos-ameacas-afirma-china-em-resposta-a-intimidacao-de-trump-com-tarifas-adicionais-de-50/', 'https://www.brasildefato.com.br/2025/04/07/tarifas-de-trump-nao-reduzirao-deficit-de-balanca-comercial-dos-eua-diz-economista-chines/', 'https://www.brasildefato.com.br/2025/04/07/trump-ameaca-a-china-com-tarifas-extras-de-50-se-o-pais-nao-retirar-as-taxas-retaliatorias-de-34/', 'https://

In [44]:
def extrai_dados_noticia_brasil_de_fato(url):
    user_agents = [
        "Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:122.0) Gecko/20100101 Firefox/122.0",
        "Mozilla/5.0 (Macintosh; Apple M1; Mac OS X 13_4) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/122.0.0.0 Safari/537.36",
        "Mozilla/5.0 (X11; Fedora; Linux x86_64; rv:124.0) Gecko/20100101 Firefox/124.0",
        "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/121.0.0.0 Safari/537.36 Edg/121.0.2277.128",
        "Mozilla/5.0 (Macintosh; Intel Mac OS X 13_3_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/121.0.6167.185 Safari/537.36 OPR/98.0.4759.82",
        "Mozilla/5.0 (iPad; CPU OS 17_2 like Mac OS X) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/17.0 Mobile/15E148 Safari/604.1",
        "Mozilla/5.0 (Linux; Android 14; Pixel 7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/123.0.6312.107 Mobile Safari/537.36",
        "Mozilla/5.0 (Linux; Android 14; SM-S911B) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/123.0.6312.86 Mobile Safari/537.36",
        "Mozilla/5.0 (iPhone; CPU iPhone OS 17_3_1 like Mac OS X) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/17.0 Mobile/15E148 Safari/604.1",
        "Mozilla/5.0 (Linux; Android 13; ONEPLUS A6013) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/123.0.6312.107 Mobile Safari/537.36",
        "Mozilla/5.0 (X11; CrOS x86_64 15393.44.0) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/123.0.6312.86 Safari/537.36",
        "Mozilla/5.0 (Android 14; Mobile; rv:124.0) Gecko/124.0 Firefox/124.0",
    ]

    headers = {
        "User-Agent": random.choice(user_agents),
        "Accept-Language": "pt-BR,pt;q=0.9",
        "Accept-Encoding": "gzip, deflate",
        "Connection": "keep-alive"
    }

    response = requests.get(url, headers=headers)
    response.encoding = response.apparent_encoding
    soup = BeautifulSoup(response.text, "html.parser")

    # Título
    titulo_tag = soup.find("h1")
    titulo = titulo_tag.get_text(strip=True) if titulo_tag else "Título não encontrado"

    # Data de publicação atualizada
    data_tag = soup.find("div", class_="jeg_meta_date")
    if data_tag:
        data_texto = data_tag.get_text(strip=True)
        data_publicacao = data_texto.replace("Publicado em", "").strip()
    else:
        data_publicacao = "Data não encontrada"

    # Conteúdo principal atualizado
    conteudo_tag = soup.find("div", class_="content-inner")
    if conteudo_tag:
        paragrafos = conteudo_tag.find_all("p")
        conteudo = "\n".join(p.get_text(strip=True) for p in paragrafos if p.get_text(strip=True))
    else:
        conteudo = "Conteúdo não encontrado"

    return {
        "titulo": titulo,
        "data_publicacao": data_publicacao,
        "conteudo": conteudo
    }

In [41]:
extrai_dados_noticia_brasil_de_fato("https://www.brasildefato.com.br/2025/04/22/papa-francisco-acompanhou-de-perto-os-desdobramentos-do-crime-da-vale-em-brumadinho-mg/")

{'titulo': 'Papa Francisco acompanhou de perto os desdobramentos do crime da Vale em Brumadinho (MG)',
 'data_publicacao': '22.abr.2025 às 18h15',
 'conteudo': 'A morte do Papa Francisco, o primeiro nascido na América Latina, na segunda-feira (21), aos 88 anos de idade, emocionou milhões de seguidores e não seguidores da Igreja Católica. O fim do pontificado do jesuíta argentino fez com que seus apoiadores relembrassem alguns de seus feitos marcantes. Entre eles, ganharam destaque as homenagens do pontífice aos atingidos pelo rompimento da barragem da Vale em Brumadinho, Minas Gerais.\nEm 2019, logo após o crime que deixou 272 vítimas fatais em janeiro daquele ano, Francisco enviou um representante do Vaticano ao município da Região Metropolitana de Belo Horizonte (RMBH). Além disso, acompanhou de perto os desdobramentos e recebeu pessoas atingidas pelo rompimento.\nNos anos seguintes, o pontífice continuou relembrando os impactos da tragédia-crime.\n“Neste aniversário de um ano da tra

In [46]:
### Estrutura os dados em dicionários ###
e_dicionario = dict()
for e in tqdm(list(d.keys()), desc="Chave"):
    f_dicionario = dict()
    for f in list(d.values())[0]:
        print(f"Extraindo dados de: {f}")
        f_dicionario[f] = extrai_dados_noticia_brasil_de_fato(f)
    e_dicionario[e] = f_dicionario

Chave:   0%|                                                                                     | 0/1 [00:00<?, ?it/s]

Extraindo dados de: https://www.brasildefato.com.br/2025/04/14/guerra-comercial-russia-tem-riscos-e-oportunidades-com-tarifas-de-trump/
Extraindo dados de: https://www.brasildefato.com.br/2025/04/13/china-pede-fim-de-tarifas-reciprocas-de-trump-eua-dao-prazo-para-inicio-de-taxas-sobre-semicondutores/
Extraindo dados de: https://www.brasildefato.com.br/2025/04/10/venezuela-aprova-decreto-de-emergencia-economica-para-combater-tarifas-de-trump/
Extraindo dados de: https://www.brasildefato.com.br/2025/04/09/lula-critica-tarifas-arbitrarias-de-trump-em-discurso-na-celac-guerras-comerciais-nao-tem-vencedores/
Extraindo dados de: https://www.brasildefato.com.br/2025/04/08/nao-aceitaremos-ameacas-afirma-china-em-resposta-a-intimidacao-de-trump-com-tarifas-adicionais-de-50/
Extraindo dados de: https://www.brasildefato.com.br/2025/04/07/tarifas-de-trump-nao-reduzirao-deficit-de-balanca-comercial-dos-eua-diz-economista-chines/
Extraindo dados de: https://www.brasildefato.com.br/2025/04/07/trump-a

Chave: 100%|████████████████████████████████████████████████████████████████████████████| 1/1 [04:47<00:00, 287.78s/it]


In [47]:
# Transformar e_dicionario em uma lista de linhas para o DataFrame
linhas = []
for chave_principal, sub_dicionario in e_dicionario.items():
    for url, dados in sub_dicionario.items():
        linha = {
            "Palavra-chave": chave_principal,
            "URL": url,
            "Título": dados.get("titulo", "Título não encontrado"),
            "Data de Publicação": dados.get("data_publicacao", "Data não encontrada"),
            "Conteúdo": dados.get("conteudo", "Conteúdo não encontrado"),
        }
        linhas.append(linha)

# Criar o DataFrame
df = pd.DataFrame(linhas)

# Exibir o DataFrame
print(df)

     Palavra-chave                                                URL  \
0    tarifas+trump  https://www.brasildefato.com.br/2025/04/14/gue...   
1    tarifas+trump  https://www.brasildefato.com.br/2025/04/13/chi...   
2    tarifas+trump  https://www.brasildefato.com.br/2025/04/10/ven...   
3    tarifas+trump  https://www.brasildefato.com.br/2025/04/09/lul...   
4    tarifas+trump  https://www.brasildefato.com.br/2025/04/08/nao...   
..             ...                                                ...   
141  tarifas+trump  https://www.brasildefato.com.br/2025/02/04/dan...   
142  tarifas+trump  https://www.brasildefato.com.br/2025/02/03/tal...   
143  tarifas+trump  https://www.brasildefato.com.br/2025/01/26/col...   
144  tarifas+trump  https://www.brasildefato.com.br/2025/01/23/bra...   
145  tarifas+trump  https://www.brasildefato.com.br/2025/01/13/chi...   

                                                Título  \
0    Guerra comercial: Rússia tem riscos e oportuni...   
1    Ch

In [48]:
### Impostação para o Excel ###

import openpyxl
df.to_excel("noticias_brasil_de_fato.xlsx", index=False, engine='openpyxl')

## 3º Jornal Online - CBC (Canadá)

In [92]:
def get_urls_from_cbc_by_screen_click(palavra_chave, click_x, click_y):
    # Configurando o Selenium
    chrome_options = Options()
    chrome_options.add_argument("--start-maximized")  # Maximiza para coordenadas ficarem corretas
    driver = webdriver.Chrome(options=chrome_options)
    
    try:
        query = "%20".join(palavra_chave.split())
        url = f"https://www.cbc.ca/search?q={query}&section=news"

        driver.get(url)
        time.sleep(10)  # Espera carregar o site
        
        # Rola um pouco para baixo
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(2)
        
        # Agora clicar na coordenada específica 5 vezes
        for _ in range(10):
            pyautogui.moveTo(click_x, click_y)
            pyautogui.click()
            print(f"Clicado em ({click_x}, {click_y})")
            time.sleep(random.uniform(3, 5))  # Espera para o conteúdo carregar
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
            
        # Após carregar, extrair o HTML final
        from bs4 import BeautifulSoup
        soup = BeautifulSoup(driver.page_source, "html.parser")
        
        links = set()
        for a_tag in soup.find_all("a", href=True):
            href = a_tag["href"]
            #print(href)
            if href.startswith("//www.cbc.ca/news/"):
                full_link = "https://" + href.lstrip("/")
                links.add(full_link.strip())
            elif href.startswith("/news/"):
                full_link = "https://www.cbc.ca" + href
                links.add(full_link.strip())
        
        return list(links)
    
    finally:
        driver.quit()

# Teste
# Primeiro você precisa saber onde clicar (x, y)
palavras_chave = ["Trump tariffs China trade war", "Trump tariffs European Union", "Trump tariffs Brazil"]
click_x = 487  # exemplo - ajuste depois
click_y = 207  # exemplo - ajuste depois

fim = dict()
for palavra in palavras_chave:
    urls = get_urls_from_cbc_by_screen_click(palavra, click_x, click_y)
    fim[palavra] = urls
fim

Clicado em (487, 207)
Clicado em (487, 207)
Clicado em (487, 207)
Clicado em (487, 207)
Clicado em (487, 207)
Clicado em (487, 207)
Clicado em (487, 207)
Clicado em (487, 207)
Clicado em (487, 207)
Clicado em (487, 207)
Clicado em (487, 207)
Clicado em (487, 207)
Clicado em (487, 207)
Clicado em (487, 207)
Clicado em (487, 207)
Clicado em (487, 207)
Clicado em (487, 207)
Clicado em (487, 207)
Clicado em (487, 207)
Clicado em (487, 207)
Clicado em (487, 207)
Clicado em (487, 207)
Clicado em (487, 207)
Clicado em (487, 207)
Clicado em (487, 207)
Clicado em (487, 207)
Clicado em (487, 207)
Clicado em (487, 207)
Clicado em (487, 207)
Clicado em (487, 207)


{'Trump tariffs China trade war': ['https://www.cbc.ca/news/canada/canada-china-electric-vehicles-1.7486204',
  'https://www.cbc.ca/news/world/china-tariffs-us-trump-1.7507719',
  'https://www.cbc.ca/news/world/chinese-sellers-amazon-us-tariffs-1.7506519',
  'https://www.cbc.ca/news/world/us-china-trade-war-1.7512114',
  'https://www.cbc.ca/news/world/trump-china-tariffs-southeast-asia-xi-1.7511681',
  'https://www.cbc.ca/news/canada/manitoba/canada-us-travel-trump-trade-war-1.7487758',
  'https://www.cbc.ca/news/canada/manitoba/manitoba-agricultural-sector-supports-trade-war-agristability-1.7500249',
  'https://www.cbc.ca/news/business/stock-markets-thursday-1.7506773',
  'https://www.cbc.ca/news/canada/british-columbia/trump-tariffs-bc-1.7500636',
  'https://www.cbc.ca/news/politics/liberal-platform-promises-130b-in-new-spending-over-4-years-adding-225b-to-federal-debt-1.7514272',
  'https://www.cbc.ca/news/canada/british-columbia/u-s-travel-trade-war-1.7482967',
  'https://www.cbc.c

In [93]:
def extrai_dados_noticia(url):
    user_agents = [
        "Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:122.0) Gecko/20100101 Firefox/122.0",
        "Mozilla/5.0 (Macintosh; Apple M1; Mac OS X 13_4) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/122.0.0.0 Safari/537.36",
        "Mozilla/5.0 (X11; Fedora; Linux x86_64; rv:124.0) Gecko/20100101 Firefox/124.0",
        "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/121.0.0.0 Safari/537.36 Edg/121.0.2277.128",
        "Mozilla/5.0 (Macintosh; Intel Mac OS X 13_3_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/121.0.6167.185 Safari/537.36 OPR/98.0.4759.82",
        "Mozilla/5.0 (iPad; CPU OS 17_2 like Mac OS X) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/17.0 Mobile/15E148 Safari/604.1",
        "Mozilla/5.0 (Linux; Android 14; Pixel 7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/123.0.6312.107 Mobile Safari/537.36",
        "Mozilla/5.0 (Linux; Android 14; SM-S911B) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/123.0.6312.86 Mobile Safari/537.36",
        "Mozilla/5.0 (iPhone; CPU iPhone OS 17_3_1 like Mac OS X) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/17.0 Mobile/15E148 Safari/604.1",
        "Mozilla/5.0 (Linux; Android 13; ONEPLUS A6013) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/123.0.6312.107 Mobile Safari/537.36",
        "Mozilla/5.0 (X11; CrOS x86_64 15393.44.0) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/123.0.6312.86 Safari/537.36",
        "Mozilla/5.0 (Android 14; Mobile; rv:124.0) Gecko/124.0 Firefox/124.0",
    ]

    headers = {
        "User-Agent": random.choice(user_agents),
        "Connection": "keep-alive"
    }

    response = requests.get(url, headers=headers)

    soup = BeautifulSoup(response.text, "html.parser")

    # Título
    titulo_tag = soup.find("h1", class_="detailHeadline")
    print(titulo_tag)
    titulo = titulo_tag.get_text(strip=True) if titulo_tag else "Título não encontrado"

    # Data de publicação atualizada
    data_tag = soup.find("time", class_="timeStamp")
    if data_tag and data_tag.has_attr('datetime'):
        datetime_value = data_tag['datetime']
        formatted_date = dt.datetime.strptime(datetime_value, '%Y-%m-%dT%H:%M:%S.%fZ').strftime('%d/%m/%Y')
        data_publicacao = formatted_date
    else:
        data_publicacao = "Data não encontrada"

    # Conteúdo principal atualizado
    conteudo_tag = soup.find("div", class_="story")
    if conteudo_tag:
        paragrafos = conteudo_tag.find_all("p")
        conteudo = "\n".join(p.get_text(strip=True) for p in paragrafos if p.get_text(strip=True))
    else:
        conteudo = "Conteúdo não encontrado"

    return {
        "titulo": titulo,
        "data_publicacao": data_publicacao,
        "conteudo": conteudo
    }


In [94]:
### Estrutura os dados em dicionários ###
cbc_dicionario = dict()
for b in list(fim.keys()):
    c_dicionario = dict()
    for c in list(fim.values())[0]:
        c_dicionario[c] = extrai_dados_noticia(c)
    cbc_dicionario[b] = c_dicionario


<h1 class="detailHeadline" data-cy="detailHeadline" lang="en">Should Canada ease its 100% tariff on electric vehicles from China amid trade war with U.S.?</h1>
<h1 class="detailHeadline" data-cy="detailHeadline" lang="en">China hits back at Trump's tariff hike with 125% duties on U.S. goods</h1>
<h1 class="detailHeadline" data-cy="detailHeadline" lang="en">Chinese Amazon sellers to hike prices or exit U.S. market as tariffs soar, association says</h1>
<h1 class="detailHeadline" data-cy="detailHeadline" lang="en">The U.S.-China trade war is escalating. But China may have more leverage than the U.S. thought</h1>
<h1 class="detailHeadline" data-cy="detailHeadline" lang="en">Chinese president engages in charm offensive in Southeast Asia as U.S. tariffs rattle region</h1>
<h1 class="detailHeadline" data-cy="detailHeadline" lang="en">Manitoba travel to U.S. takes hit amid Trump's tariff war</h1>
<h1 class="detailHeadline" data-cy="detailHeadline" lang="en">Manitoba to match federal agricultu

In [96]:

# Transformar b_dicionario em uma lista de linhas para o DataFrame
linhas = []
for chave_principal, sub_dicionario in cbc_dicionario.items():
    for url, dados in sub_dicionario.items():
        linha = {
            "Palavra-chave": chave_principal,
            "URL": url,
            "Título": dados.get("titulo", "Título não encontrado"),
            "Data de Publicação": dados.get("data_publicacao", "Data não encontrada"),
            "Conteúdo": dados.get("conteudo", "Conteúdo não encontrado"),
        }
        linhas.append(linha)

# Criar o DataFrame
df = pd.DataFrame(linhas)
df

,Palavra-chave,URL,Título,Data de Publicação,Conteúdo
0,Trump tariffs China trade war,https://www.cbc.ca/news/canada/canada-china-el...,Should Canada ease its 100% tariff on electric...,20/03/2025,Canada's trade war with the United States has ...
1,Trump tariffs China trade war,https://www.cbc.ca/news/world/china-tariffs-us...,China hits back at Trump's tariff hike with 12...,11/04/2025,Beijing on Friday increased its tariffs on U.S...
2,Trump tariffs China trade war,https://www.cbc.ca/news/world/chinese-sellers-...,Chinese Amazon sellers to hike prices or exit ...,10/04/2025,Chinese companies that sell products on Amazon...
3,Trump tariffs China trade war,https://www.cbc.ca/news/world/us-china-trade-w...,The U.S.-China trade war is escalating. But Ch...,17/04/2025,As the trade war between the U.S. and China es...
4,Trump tariffs China trade war,https://www.cbc.ca/news/world/trump-china-tari...,Chinese president engages in charm offensive i...,18/04/2025,The timing couldn't be better for Chinese Pres...
...,...,...,...,...,...
112,Trump tariffs Brazil,https://www.cbc.ca/news/canada/toronto/ford-us...,"Ontario rips up Starlink deal, plans to tax el...",05/03/2025,Ontario will rip up its $100-million deal with...
113,Trump tariffs Brazil,https://www.cbc.ca/news/canada/newfoundland-la...,Auto part retailers in N.L. have been preparin...,08/04/2025,While the Ontario auto sector might be the tar...
114,Trump tariffs Brazil,https://www.cbc.ca/news/canada/new-brunswick/h...,N.B. may have dodged bullet with Trump tariff ...,02/04/2025,Just two hours after U.S. President Donald Tru...
115,Trump tariffs Brazil,https://www.cbc.ca/news/canada/saskatchewan/tr...,"Uncertainty reigns as Sask. farmers, business ...",03/04/2025,"Saskatchewan farmers, business leaders and tra..."


In [43]:
import pyautogui
import time

print("O mouse vai se mover em 3 segundos...")
time.sleep(3)

pyautogui.moveTo(487, 200, duration=1)
pyautogui.moveTo(487, 205, duration=1)
pyautogui.moveTo(487, 215, duration=1)  # Move para (500, 500) em 1 segundo
#pyautogui.click()
print("Mouse movido e clicado!")


O mouse vai se mover em 3 segundos...
Mouse movido e clicado!


In [28]:
import pyautogui
import time

print("Mova o mouse para o local desejado. Pressione CTRL+C para parar.")

try:
    while True:
        x, y = pyautogui.position()
        print(f"Posição atual: X={x}, Y={y}", end="\r")  # Atualiza na mesma linha
        time.sleep(0.1)
except KeyboardInterrupt:
    print("\nPrograma finalizado.")

Mova o mouse para o local desejado. Pressione CTRL+C para parar.
Posição atual: X=665, Y=1069
Programa finalizado.


In [58]:
urls[0]

'https://www.cbc.ca/news/canada/sudbury/minecraft-server-autism-community-1.7354082'

In [98]:
#cbc_dicionario

In [ ]:
import openpyxl
#df.to_excel("noticias_CBC.xlsx", index=False, engine='openpyxl')